In [1]:
import sys
sys.path.append('../inputs/')
sys.path.append('../examples/')
from sanDiego import *
import cantera as ct

# <font color='brown'> Introduction

<font size=3>

This notebook illustrates how to use Prometheus thermochemistry kernels (or $\texttt{PyPrometheus}$-generated source code, and simply abbreviated as PTKs) to compute chemical source terms. Chemical source terms are models for atom re-shuffling and heat release by chemical reactions, and they appear in the species transport equations
\begin{equation}
    \frac{ \partial \rho Y_{i} }{ \partial t } + \cdots = \underbrace{ W_{i}\omega_{i} }_{S_{i}},
\end{equation}
where $\rho Y_{i}$ is the density of the $i^{\mathrm{th}}$ species, $S_{i}$ its chemical source term, $W_{i}$ its molecular weight, and $\omega_{i}$ its net production rate. We will not delve here in how the code is generated. For that, a separate notebook will be compiled. 

First, note that the very first cell imports everything from $\texttt{sanDiego.py}$, which contains Python code to evaluate chemical source terms based on the (hydrogen combustion) San Diego mechanism. At this early stage, it is the only model that has been implemented with $\texttt{PyPrometheus}$. This is likely to change (or, better put, be generalized), as more models get implemented. We have also imported Cantera for useful comparisons and verification.

With everything that we need loaded, the next thing to do is create an instance of the thermochemistry kernel. This is a one-liner, and is done in the next cell.

In [2]:
pt = prometheus_thermochemistry_kernel()
print('Working with prometheus thermochemistry kernel with model name %s' % pt.model_name)

Working with prometheus thermochemistry kernel with model name sanDiego


### <font color='brown'> Computing Species Source Terms

<font size=3>

Most likely what you are here to learn is: how do we use this thermochemistry kernel in a compressible code? In such situation, the mixture composition will be given in terms of the density $\rho$ (in $\mathrm{k/m^{3}}$), the internal energy $e$ (in $\mathrm{J/kg}$), and the species mass fractions $\{ Y_{i} \}_{i = 1}^{N}$, where $N$ is the number of species. Let's start by assigning these state variables some (merely illustrative) values.

In [3]:
# density [kg/m^3]
rho   = 0.2040 
# internal energy [J/kg]
e_int = 2.3624875e7
# mass fractions
N  = pt.num_species
Y  = np.ones( N ) / float( N )

<font size=3>

An immediate challenge is that species production rates $\{ \dot{\omega}_{i} \}_{i = 1}^{N}$ (in $\mathrm{kmol/m^{3}-s}$) are typically (and naturally) provided in terms of the temperature $T$ (in $\mathrm{K}$) instead of the interal energy $e$. However, the temperature can easily be backed out from
\begin{equation}
    e = \sum_{i = 1}^{N}e_{i}(T)Y_{i},
\end{equation}
where $\{ e_{i}(T) \}_{i = 1}^{N}$ are the species internal energies. PTKs provide a function (a Newton method) for inverting this algebraic equation.

In [4]:
T_guess = 320.0 # [K]
T = pt.get_temperature( e_int, T_guess, Y, do_energy = True )
print('The temperature of the mixture is %.2f K ' % T)

The temperature of the mixture is 300.00 K 


<font size=3>

This function takes four input arguments. From right to left (for clarity): 
4. $\texttt{do}\_\texttt{energy}$, specifying whether we want to compute the temperature from the internal energy ($\texttt{True}$) or the enthalpy ($\texttt{False}$); 
3. $\texttt{Y}$, the mass fractions; 
2. $\texttt{T}\_\texttt{guess}$, an initial guess to the Newton iteration; 
1. $\texttt{e}\_\texttt{int}$, the internal energy (or enthalpy, if $\texttt{do}\_\texttt{energy = False}$)

Knowing the temperature, we can compute the species net production rates.

In [5]:
omega = pt.get_net_production_rates( rho, T, Y )

<font size=3>

In a flow solver, we would multiply these production rates by the molecular weights $\{ W_{i} \}_{i = 1}^{N}$ (in $\mathrm{kg/kmol}$) and add them to the RHS. In any case, we have what we came for$\dots$ but how do we know it's right? While full verification of PTKs is covered in another notebook, we can easily gain confidence that what we've done so far is correct. For this, we use Cantera as a reference. (Note: the Cantera solution object is created from the same mechanism file that was used to generate the PTK.) The metric by which we verify our source terms is the relative error
\begin{equation}
    \varepsilon_{i} = \Bigg|\frac{ \omega_{i,\texttt{ct}}-\omega_{i,\texttt{ptk}} }{ \omega_{i,\texttt{ct}} }\Bigg|,
\end{equation}
where $\omega_{i,\texttt{ct}}$ and $\omega_{i,\texttt{ptk}}$ are the source terms computed using Cantera and the PTK. The largest relative error is $\sim 0.003\%$, which is satisfactory. 


In [8]:
# Need pressure to set state in Cantera
pressure = pt.get_pressure( rho, T, Y )
# Create a new solution, consistent with PTK
ct_gas = ct.Solution('../inputs/sanDiego.cti','gas')
# Set state and get production rates
ct_gas.TPY = T, pressure, Y
ct_species = ct_gas.species_names
ct_omega   = ct_gas.net_production_rates


for i in range(0,N-1):
    rel_err = np.abs( (ct_omega[i] - omega[i]) / ct_omega[i]  )
    print('[%s] \t Relative error: %.5f%%'% ( ct_species[i], 100.0 * rel_err ) )
    
print('[N2] \t omega_ct = %.3f, omega_ptk = %.3f' % ( ct_omega[-1], omega[-1] ) )

[H2] 	 Relative error: 0.00039%
[H] 	 Relative error: 0.00012%
[O2] 	 Relative error: 0.00011%
[O] 	 Relative error: 0.00043%
[OH] 	 Relative error: 0.00338%
[HO2] 	 Relative error: 0.00042%
[H2O2] 	 Relative error: 0.00012%
[H2O] 	 Relative error: 0.00036%
[N2] 	 omega_ct = 0.000, omega_ptk = 0.000
